## 文档的创建和实例

In [2]:
from mongoengine import connect, disconnect
from mongoengine import Document
from mongoengine import StringField, IntField
from mongo_config import TEST_DB1, TEST_DB2, TEST_DB3, HOST, PORT, USERNAME, PASSWORD

In [3]:
from mongoengine import BooleanField, DateTimeField, DictField, EmailField, FloatField, ListField, ReferenceField

In [4]:
# 连接到已创建的数据库
connect(TEST_DB1, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB1, alias=TEST_DB1)
connect(TEST_DB2, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB2, alias=TEST_DB2)
connect(TEST_DB3, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB3, alias=TEST_DB3)

MongoClient(host=['192.168.2.172:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [3]:
# 继承自Document的文档
class Book1(Document):
    bookid = StringField()
    name = StringField()
    count = IntField()
    meta = {"db_alias": TEST_DB1}
Book1(bookid="wx000006", name="水浒传", count=29).save()

In [6]:
# 动态文档, 可以在原有的数据模式上动态添加字段
from mongoengine import DynamicDocument
class BookDynamic(DynamicDocument):
    bookid = StringField()
    meta = {"db_alias": TEST_DB1}
bookd = BookDynamic(bookid="wx09872")
bookd.tag = ["小说", "经典著作"]
bookd.save()

<BookDynamic: BookDynamic object>

### 一些主要并常用的字段类型

In [5]:
class BookType(Document):
    type_name = StringField()
    meta = {"db_alias": TEST_DB1}

In [17]:
BookType(type_name="经济类").save()
BookType(type_name="文学类").save()
BookType(type_name="计算机").save()

<BookType: BookType object>

In [6]:
class BookAuthor(Document):
    author_name = StringField()
    meta = {"db_alias": TEST_DB1}

In [7]:
class BookAllField(Document):
    bookid = StringField()
    name = StringField()
    ifnewbook = BooleanField()  # 是否为新上架图书
    shelves_date = DateTimeField()
    author = ReferenceField(BookAuthor)  # 使引用字段执行其他集合
    author_email = EmailField()
    price = FloatField()
    book_type = ReferenceField(BookType)
    tag = ListField()
    meta = {"db_alias": TEST_DB1}

In [29]:
book_all = BookAllField(bookid="wx092761")

In [30]:
# 包含使用引用字段的情形
book_all.name = "史记"
book_all.ifnewbook = False
book_all.author = BookAuthor(author_name="司马迁").save()  # 此时的引用可以成功
book_all.price = 19.8
# book_all.book_type = BookType(type_name="文学类")  # 此时不能引用，需要有保存动作，或者查询出已经保存的项目
# book_all.book_type = BookType.objects(type_name="文学类") # 没保存成功
book_all.tag = ["历史", "经典"]
book_all.save()

<BookAllField: BookAllField object>

### EmailField-邮箱字段

> classmongoengine.fields.EmailField(domain_whitelist=None, allow_utf8_user=False, allow_ip_domain=False, *args, **kwargs)

    A field that validates input as an email address.

    Parameters
    - domain_whitelist – (optional) list of valid domain names applied during validation
    - allow_utf8_user – Allow user part of the email to contain utf8 char
    - allow_ip_domain – Allow domain part of the email to be an IPv4 or IPv6 address
    - kwargs – Keyword arguments passed into the parent StringField

因此，此字段能够验证和筛选邮箱。不用在StringField字段下添加筛选验证函数。

In [8]:
book_e = BookAllField(bookid="wx09sah61", name = "本草纲目", ifnewbook=False, price = 19.8)
book_e.author = BookAuthor(author_name="李时珍").save()  # 此时的引用可以成功
book_e.book_type = BookType(type_name="医学") .save()  # 实际上每次保存会产生大量的重复数据
# tag = ListField() ，没有赋值；那么默认值是 空列表，即在数据库中显示 [];虽然此时 ListField() 中没有使用default参数

In [12]:
book_e.author_email = "lsz@qq.com"

In [10]:
book_e.author_email = "qq.com"
# 使用此赋值时，会报错
# ValidationError: ValidationError (BookAllField:None) (Invalid email address: qq.com: ['author_email'])

In [14]:
book_e.save()

<BookAllField: BookAllField object>

正确的保存结果:
```json
{
    "_id" : ObjectId("63fe9de15990799fbfd9fb90"),
    "bookid" : "wx09sah61",
    "name" : "本草纲目",
    "ifnewbook" : false,
    "author" : ObjectId("63fe9da75990799fbfd9fb8e"),
    "author_email" : "lsz@qq.com",
    "price" : 19.8,
    "book_type" : ObjectId("63fe9da75990799fbfd9fb8f"),
    "tag" : []
}
```